In [2]:
#!pip3 install neo4j-driver
#!pip3 install py2neo
#!pip3 install neo4j-driver===1.6.2

In [3]:
from neo4j import *
from py2neo import *
from pymongo import MongoClient

In [4]:
# http://localhost:7474
# default password: neo4j
# updated password: neo4j2

# https://neo4j.com/docs/cypher-refcard/3.2/
# https://neo4j.com/docs/cypher-manual/3.5/

In [5]:
db = Database("http://localhost:7474/db/data/", user="neo4j", password="neo4j2")

In [6]:
from py2neo import Graph, Path
graph = Graph(password="neo4j2")

In [7]:
client = MongoClient('localhost', 27017)

In [8]:
db = client.jazz_catalog

In [9]:
db.artists.count()

1504

In [10]:
# use MERGE for unique

for artist in db.artists.find():
    graph.run("MERGE (artist:Artist {name:{name}, id:{id}}) RETURN artist", name=artist['name'], id=artist['id'])

In [11]:
c = graph.run("MATCH (n:Artist) WHERE n.name = 'Miles Davis' RETURN n.id")

In [12]:
for i in c:
    print(i)

<Record n.id='561d854a-6a28-4aa7-8c99-323e6ce46c2a'>


In [13]:
# MERGE (instrument:Instrument {name: 'trumpet'}) RETURN instrument.name
# MATCH (a:Artist),(b:Instrument) 
# WHERE a.name = 'Miles Davis' AND b.name = 'trumpet' MERGE (a)-[r:PLAYS { name: a.name + '->' + b.name }]->(b) RETURN type(r), r.name

In [14]:
# set instruments for artists
# create node for instruments and add 'plays' relationship

run_merge = "MERGE (instrument:Instrument {name: {name}}) RETURN instrument.name"
run_match = "MATCH (a:Artist), (b:Instrument) WHERE a.id = {aid} AND b.name = {name} MERGE (a)-[r:PLAYS {name: a.name + '->' + b.name}]->(b)"

for r in db.releases.find():
    if 'medium-list' in r:
        media = r['medium-list']
        for m in media:
            if 'track-list' in m:
                tracks = m['track-list']
                for each in tracks:
                    if 'recording' in each.keys():
                        if 'artist-relation-list' in each['recording'].keys():
                            artist_list = each['recording']['artist-relation-list']
                            for a in artist_list:
                                artist_id = a['artist']['id']
                                if 'attribute-list' in a.keys():
                                    instruments = a['attribute-list']
                                    for inst in instruments:
                                        graph.run(run_merge, name=inst)
                                        graph.run(run_match, aid=artist_id, name=inst)
                                

## CRUD Operations

The basic operations can be demonstrated in the Neo4j browser: [http://localhost:7474/browser/](http://localhost:7474/browser/).

**Create**

Neo4j's _Cypher_ query language provides a CREATE clause, but we use the MERGE statement to enforce uniqueness.

```
MERGE (artist:Artist {name: 'Billie Holiday', id: 'd59c4cda-11d9-48db-8bfe-b557ee602aed'}) RETURN artist

MERGE (instrument:Instrument {name: 'voice'}) RETURN instrument

MATCH (a:Artist), (b:Instrument) WHERE a.name = 'Billie Holiday' AND b.name = 'voice' MERGE (a)-[r:PLAYS {name: a.name + '->' + b.name}]->(b)"
```

**Retrieve**

```
MATCH (n) RETURN n LIMIT 100

MATCH (n:Instrument) RETURN n LIMIT 100

MATCH (:Artist { name: 'Billie Holiday' })--(instrument) RETURN instrument.name

MATCH (:Artist { name: 'Ron Carter' })--(instrument) RETURN instrument.name
```

That last query in pseudo-SQL:

```
SELECT artist.name, instrument.name
FROM artist
INNER JOIN instrument ON artist.id = instrument.artist_id
WHERE artist.name == 'Ron Carter'
```

Retrieve relationships:

```
MATCH (:Artist { name: 'Billie Holiday' })-[r]->(instrument) RETURN type(r)

MATCH (:Artist { name: 'Ron Carter' })-[r]->(instrument) RETURN type(r)

MATCH p=(:Artist { name: 'Billie Holiday' })-[r:PLAYS]->() RETURN p

MATCH p=(:Artist { name: 'Ron Carter' })-[r:PLAYS]->() RETURN p

MATCH p=()-[r:PLAYS]->(:Instrument { name: 'bass' }) RETURN p LIMIT 50
```

**Update**

